In [36]:
import pandas_gbq

project_id = "cp-cdo-prod-ap-self-service"

sql = """
SELECT bic_cust_h03,bic_zcategry,bic_zsubcats,bic_zsubbr,ppgtext,SLE_Actual_LE,VersionID,Month,Key_Figure, SUM(keyfigurevalue) value
FROM `cp-cdo-prod-ap-self-service.1_SELF_SERVICE_DATASETS_AP.TBL_PNL_CALC_AP` 
WHERE Sales_Org_Value = "TH01"
AND SLE_Actual_LE IN ("2021 Actuals", "2022 Actuals (Daily YTD)")
AND Key_Figure IN ("Total Volume (Sales UoM)", "Total Tons", "Std Cases")
AND bic_zsubbr = "Care Talc Powder"
AND Month = '10'
GROUP BY bic_cust_h03,bic_zcategry,bic_zsubcats,bic_zsubbr,SLE_Actual_LE,VersionID,Month,Key_Figure, ppgtext
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████| 486/486 [00:00<00:00, 2202.71rows/s]


In [12]:
df

,bic_cust_h03,bic_zcategry,bic_zsubcats,bic_zsubbr,SLE_Actual_LE,VersionID,Month,Key_Figure,value
0,CJ EXPRESS GROUP CPG,Personal Care,Talcum Powder,Care Talc Powder,2022 Actuals (Daily YTD),Live,10,Total Tons,15.637
1,MAKRO CPG,Personal Care,Talcum Powder,Care Talc Powder,2022 Actuals (Daily YTD),Live,10,Total Tons,71.804
2,7 ELEVEN CPG,Personal Care,Talcum Powder,Care Talc Powder,2021 Actuals,Live,10,Std Cases,9828.000
3,DISTRIBUTORS CPG,Personal Care,Talcum Powder,Care Talc Powder,2022 Actuals (Daily YTD),Live,10,Total Tons,138.891
4,MAKRO CPG,Personal Care,Talcum Powder,Care Talc Powder,2022 Actuals (Daily YTD),Live,10,Std Cases,16518.000
...,...,...,...,...,...,...,...,...,...
91,FAMILY MART CPG,Personal Care,Talcum Powder,Care Talc Powder,2021 Actuals,Live,10,Total Volume (Sales UoM),177.000
92,CJ EXPRESS GROUP CPG,Personal Care,Talcum Powder,Care Talc Powder,2022 Actuals (Daily YTD),Live,10,Std Cases,2689.000
93,DISTRIBUTORS CPG,Personal Care,Talcum Powder,Care Talc Powder,2021 Actuals,Live,10,Total Volume (Sales UoM),35276.000
94,MAKRO CPG,Personal Care,Talcum Powder,Care Talc Powder,2022 Actuals (Daily YTD),Live,10,Total Volume (Sales UoM),16518.000


#By Customer

In [46]:
df.loc[df['Key_Figure'] == "Total Volume (Sales UoM)", 'Key_Figure'] = 'Volume'
result_ach = (
    df
    [['bic_cust_h03','SLE_Actual_LE','VersionID','Key_Figure', 'value']]
    .query("Key_Figure in ['Volume','Total Tons','Std Cases']")
    .assign(VersionID= lambda x: ['2022' if i == "2022 Actuals (Daily YTD)" else '2021' for i in x['SLE_Actual_LE']])
    .groupby(['bic_cust_h03','VersionID','Key_Figure'])['value']
    .agg('sum')
    .unstack(-1)
    .unstack(-1)
    .stack(0)
    .pipe(lambda x: x.assign(**{'%Ach': x['2022'] / x['2021']}))
    .unstack()
    .swaplevel(axis=1)
)



new_index = ['TESCO CPG','BIG C CPG','TOPS CPG', 'THE MALL CPG',
             'CJ EXPRESS GROUP CPG','7 ELEVEN CPG','FAMILY MART CPG', 
             'AEON CPG',"WATSON'S CPG","BOOTS CPG", "DISTRIBUTORS CPG", 
             "INDEPENDENT WHOLSALE/C&C CPG","INDEPENDENT SUPERMARKET CPG","MAKRO CPG",
             'E-Commerce-eDistributor', "PROFESSIONAL CPG", "INDOCHINA - OTHER CPG", 
             "MYANMAR CPG", "LAOS CPG", "CAMBODIA CPG", "EXPORT - COLGATE SUBSIDARIES"]
#@title
result_ach = result_ach.reindex(new_index)

result_ach = result_ach[['Volume', 'Total Tons', 'Std Cases']]
s= result_ach.style.format(thousands=",",na_rep='-',
                    formatter={('Volume', '2022'): "{:,.0f}",
                               ('Volume', '2021'): "{:,.0f}",
                               ('Volume', '%Ach'): "{:.1%}",
                               ('Total Tons', '2022'): "{:,.0f}",
                               ('Total Tons', '2021'): "{:,.0f}",
                               ('Total Tons', '%Ach'): "{:.1%}",
                               ('Std Cases', '2022'): "{:,.0f}",
                               ('Std Cases', '2021'): "{:,.0f}",
                               ('Std Cases', '%Ach'): "{:.1%}"})

In [45]:
s

# By PPGs

In [42]:
df.loc[df['Key_Figure'] == "Total Volume (Sales UoM)", 'Key_Figure'] = 'Volume'
result_ach = (
    df
    [['bic_cust_h03','ppgtext','SLE_Actual_LE','VersionID','Key_Figure', 'value']]
    .query("Key_Figure in ['Volume','Total Tons','Std Cases']")
    .assign(VersionID= lambda x: ['2022' if i == "2022 Actuals (Daily YTD)" else '2021' for i in x['SLE_Actual_LE']])
    .groupby(['ppgtext','VersionID','Key_Figure'])['value']
    .agg('sum')
    .unstack(-1)
    .unstack(-1)
    .stack(0)
    .pipe(lambda x: x.assign(**{'%Ach': x['2022'] / x['2021']}))
    .unstack()
    .swaplevel(axis=1)
)

result_ach = result_ach[['Volume', 'Total Tons', 'Std Cases']]
by_ppg_df = result_ach.style.format(thousands=",",na_rep='-',
                    formatter={('Volume', '2022'): "{:,.0f}",
                               ('Volume', '2021'): "{:,.0f}",
                               ('Volume', '%Ach'): "{:.1%}",
                               ('Total Tons', '2022'): "{:,.0f}",
                               ('Total Tons', '2021'): "{:,.0f}",
                               ('Total Tons', '%Ach'): "{:.1%}",
                               ('Std Cases', '2022'): "{:,.0f}",
                               ('Std Cases', '2021'): "{:,.0f}",
                               ('Std Cases', '%Ach'): "{:.1%}"})

In [44]:
by_ppg_df.to_excel("by_ppgs.xlsx")